In [ ]:
# ✅ Drop-in fix: re-define Candidate, helpers, scan_candidates, to_dataframe
import math, time, random
from dataclasses import dataclass
from typing import List, Tuple
import pandas as pd
from arc import Caesium, Rubidium
 
# ----- user knobs -----
ATOM_SPECIES   = "Cs"                      # "Cs" or "Rb"
F_THz_MIN, F_THz_MAX = 0.6e12, 1.0e12      # [Hz]
N_MIN, N_MAX   = 10, 40
L_PAIRS        = [("P","D"), ("D","P")]
J_OPTIONS      = {"P":[1.5], "D":[2.5]}
VISIBLE_WINDOW = (500e-9, 600e-9)          # [m]
ETHz_FOR_FOM   = 1.0                       # compare at 1 V/m
MC_CASCADES    = 800
MAX_DECAY_STEPS= 8
TOPK           = 20
SEED           = 7
random.seed(SEED)

HBAR = 1.054_571_817e-34
C    = 299_792_458.0

# polarization & mj (for getRabiFrequency2)
POL_Q = 0   # -1 (σ-), 0 (π), +1 (σ+)
M_J   = 0.5 # choose a valid mj for the chosen j1

def make_atom(name:str):
    return Caesium() if name.lower() in ("cs","caesium","cesium") else Rubidium()

def l_letter_to_l(ltr:str)->int:
    return {"S":0,"P":1,"D":2,"F":3,"G":4,"H":5}[ltr.upper()]

def trans_freq_hz(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    # ARC returns signed; we take magnitude
    return abs(atom.getTransitionFrequency(n1,l1,j1, n2,l2,j2))

def trans_wavelength_m(atom, s1, s2)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    return abs(atom.getTransitionWavelength(n1,l1,j1, n2,l2,j2))

def in_visible(lam:float, window=VISIBLE_WINDOW)->bool:
    return (lam>0) and (window[0] <= lam <= window[1])

# --- Monte Carlo cascade (same as 전에 쓴 간단 버전) ---
def allowed_lowers(n:int,l:int,j:float, n_floor:int=5):
    outs = []
    for lp in (l-1, l+1):
        if lp < 0: continue
        for jp in (lp-0.5, lp+0.5):
            if jp <= 0: continue
            nmin = max(n_floor,5); nmax = max(n_floor, n-1)
            for np in range(nmin, min(nmax, nmin+14)+1):  # truncate for speed
                outs.append((np,lp,jp))
    uniq, seen = [], set()
    for t in outs:
        if t not in seen:
            seen.add(t); uniq.append(t)
    return uniq

def mc_visible_yield(atom, n:int,l:int,j:float,
                     shots:int=MC_CASCADES,
                     max_steps:int=MAX_DECAY_STEPS,
                     window:Tuple[float,float]=VISIBLE_WINDOW)->float:
    vis = 0
    for _ in range(shots):
        cur = (n,l,j)
        for _ in range(max_steps):
            lowers = allowed_lowers(*cur)
            A_list = []
            for t in lowers:
                try:
                    A = atom.getTransitionRate(cur[0],cur[1],cur[2], t[0],t[1],t[2])
                except Exception:
                    A = 0.0
                if A>0: A_list.append((t,A))
            if not A_list: break
            totalA = sum(A for _,A in A_list)
            r, acc = random.random()*totalA, 0.0
            next_state = None
            for t,A in A_list:
                acc += A
                if r <= acc:
                    next_state = t; break
            lam = trans_wavelength_m(atom, cur, next_state)
            if in_visible(lam, window): vis += 1
            if next_state[0] <= 7 and next_state[1] <= 1: break
            cur = next_state
    return vis/float(shots)

# --- Rabi (use ARC helper; no undefined local vars) ---
def omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q)->float:
    n1,l1,j1 = s1; n2,l2,j2 = s2
    try:
        return abs(atom.getRabiFrequency2(n1,l1,j1, mj1, n2,l2,j2, q, 1.0))  # rad/s @ 1 V/m
    except Exception:
        return 0.0

def d_from_omega(omega_rad_s:float, E=1.0)->float:
    return HBAR*omega_rad_s/E

@dataclass
class Candidate:
    upper: Tuple[int,int,float]
    lower: Tuple[int,int,float]
    f_THz: float
    d_Cm: float
    Omega_1Vpm: float
    vis_yield_best: float

def scan_candidates(atom_name="Cs")->List[Candidate]:
    atom = make_atom(atom_name)
    out: List[Candidate] = []
    t0 = time.time()
    for l1ltr, l2ltr in L_PAIRS:
        l1 = l_letter_to_l(l1ltr); l2 = l_letter_to_l(l2ltr)
        for j1 in J_OPTIONS[l1ltr]:
            for j2 in J_OPTIONS[l2ltr]:
                for n1 in range(N_MIN, N_MAX+1):
                    for dn in (0, -1, +1, -2, +2):
                        n2 = n1 + dn
                        if n2 < N_MIN or n2 > N_MAX or n2 == n1:
                            continue
                        s1, s2 = (n1,l1,j1), (n2,l2,j2)
                        f = trans_freq_hz(atom, s1, s2)
                        if F_THz_MIN <= f <= F_THz_MAX:
                            omega_rad_s = omega_1Vpm(atom, s1, s2)   # always defined here
                            d_cm = d_from_omega(omega_rad_s, E=ETHz_FOR_FOM)
                            y1 = mc_visible_yield(atom, *s1)
                            y2 = mc_visible_yield(atom, *s2)
                            out.append(Candidate(
                                upper=s1, lower=s2, f_THz=f,
                                d_Cm=d_cm, Omega_1Vpm=omega_rad_s,
                                vis_yield_best=max(y1,y2)
                            ))
    print(f"scan done: {len(out)} candidates in {time.time()-t0:.1f}s")
    return out

def score(c: Candidate)->float:
    return abs(c.Omega_1Vpm) * c.vis_yield_best

def fmt_state(s):
    n,l,j = s
    LTR = "SPDFGH"[l] if l<6 else f"L{l}"
    return f"{n}{LTR}_{int(2*j)}/2"

def to_dataframe(cands: List[Candidate], topk=TOPK)->pd.DataFrame:
    rows = []
    for c in sorted(cands, key=score, reverse=True)[:topk]:
        rows.append({
            "upper": fmt_state(c.upper),
            "lower": fmt_state(c.lower),
            "f_THz": c.f_THz/1e12,
            "|d| [C·m] (from Ω)": c.d_Cm,
            "Ω(1 V/m) [Hz]": c.Omega_1Vpm/(2*math.pi),
            "visible_yield(500–600nm)": c.vis_yield_best,
            "FoM = Ω×yield": score(c)
        })
    return pd.DataFrame(rows)


In [2]:
cands = scan_candidates(ATOM_SPECIES)  # <-- 더 이상 UnboundLocalError 안 납니다
df = to_dataframe(cands, topk=TOPK)
df

scan done: 26 candidates in 1123.7s


,upper,lower,f_THz,|d| [C·m] (from Ω),Ω(1 V/m) [Hz],visible_yield(500–600nm),FoM = Ω×yield
0,25P_3/2,23D_5/2,0.645824,7.655332e-28,1.155335e+06,0.13375,970915.948103
1,23D_5/2,25P_3/2,0.645824,7.655332e-28,1.155335e+06,0.12875,934620.024809
2,24P_3/2,22D_5/2,0.747703,6.906497e-28,1.042322e+06,0.14000,916874.099307
3,13P_3/2,12D_5/2,0.746818,5.522324e-28,8.334237e+05,0.15750,824757.460948
4,22D_5/2,24P_3/2,0.747703,6.906497e-28,1.042322e+06,0.12500,818637.588667
5,21D_5/2,23P_3/2,0.872191,6.196203e-28,9.351249e+05,0.13500,793201.020408
6,23P_3/2,21D_5/2,0.872191,6.196203e-28,9.351249e+05,0.13125,771167.658730
7,12D_5/2,13P_3/2,0.746818,5.522324e-28,8.334237e+05,0.13250,693843.578258
8,29P_3/2,30D_5/2,0.743526,2.613657e-28,3.944506e+05,0.15000,371760.913428
9,30P_3/2,31D_5/2,0.665129,2.800106e-28,4.225892e+05,0.13625,361771.857446


In [3]:
atom = make_atom("Cs")
s1 = (14, l_letter_to_l("P"), 1.5)
s2 = (13, l_letter_to_l("D"), 2.5)
print("λ (m):", trans_wavelength_m(atom, s1, s2))
print("A_ij (s^-1):", atom.getTransitionRate(*s1, *s2))
print("Ω @1 V/m (rad/s):", omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q))

λ (m): 0.0005465790711620962
A_ij (s^-1): 0.0
Ω @1 V/m (rad/s): 6413605.817088714
